# Model Selection (with Hyperparameter Tuning)
Now that we have completed feature engineering, we are ready to start testing our dataset against various algorithms to see which produces the most accurate inferences. As a reminder, this project technically contains **two different models** as we are looking to predict two different values: a binary "yes/no" approval rating and a single-point decimal float between 0.0 and 10.0, more lovingly referred to as the *Biehn Scale*.

For each model we'll be creating, we will be testing **five different types of algorithms** to assess which performs the best. You might be asking the question, "Which algorithm is the best for my situation?" A super strong mathematician might be able to give you a decent answer, but at the end of the day, the truth is simply this: **The algorithm most right for your project is the one that produces the most consistently accurate results!** To that end, we test multiple algorithms instead of settling on a single one.

The goal of this notebook is to assess the results of each of the algorithms. Once we settle on one that seems to produce the best results, then we will create another notebook to formalize the model training process with a full ML pipeline.

## Project Setup
Let's go ahead and perform a handful of activities as we prepare start the model selection.

In [ ]:
# Importing the necessary Python libraries
import numpy as np
import pandas as pd